<a href="https://colab.research.google.com/github/AJ-Horch/customer_lifetime_value_prediction/blob/main/Customer_Lifetime_Value_Prediciton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Customer Lifetime Value Prediciton
Data: [Online Retail Data from UCI Repo](https://archive.ics.uci.edu/dataset/352/online+retail)

CLV is the total net revenue a company can expect from a single customer throughout their relationship
- The first objective is to calculate and predict future CLV, whic will help you find out how much each customer will spend
- The second objective is to identify profitable customers. The model will tell you who those valuable customers are
- The third goal is to take marketing actions based on the analysis from there, you will be able to optiize your marketing budget.


In [1]:
# Installing the PyMC Marketing library
!pip install pymc_marketing

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.9/253.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 8.4 MB/s eta 0:00:00


In [2]:
!pip install openpyxl

In [3]:
!pip install arviz

In [4]:
# import required libraries
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc as pm
from arviz.labels import MapLabeller

from IPython.display import Image
from pymc_marketing import clv

In [5]:
# setting the style of the plots
az.style.use('arviz-darkgrid')

%config InlineBackend.figure_format = 'retina' # good looking plots


In [8]:
import requests
import zipfile
import os

# Download the zip file
url = "https://archive.ics.uci.edu/static/public/352/online+retail.zip"
reponse = requests.get(url)
filename = 'online_retial.zip'

with open(filename, 'wb') as file:
  file.write(reponse.content)

# unzip the file
with zipfile.ZipFile(filename, 'r') as zip_ref:
  zip_ref.extractall('online_retail_data')

# finding the excel file name
for file in os.listdir('online_retail_data'):
  if file.endswith(".xlsx"):
    excel_file = os.path.join("online_retail_data", file)
    break

# convert from excel to csv
data_raw = pd.read_excel(excel_file)

data_raw.head()

BadZipFile: File is not a zip file

In [ ]:
data_raw.info()

In [ ]:
# If the InvoiceNo starts with 'c' it indicates a cancellation
cancelled_orders = data_raw[data_raw['InvoiceNo'].astype(str).str.startswith("C")]

# Create a temporary DF with the columns I want to match on. Also negate the 'Quantity col
cancelled_orders['Quantity'] = -cancelled_orders['Quantity']

# Merge the orginal DF with the temp on the columns I want to match
merged_data = pd.merge(data_raw, cancelled_orders[['CustomerID', 'StockCode', 'Quantity',
                                                   'UnitPrice']],
                       on=['CustomerID', 'StockCode', 'Quantity', 'UnitPrice'],
                       how='left', indicator=True)

# Filter out rows where the merge found a match, and also filter out the orginal return orders
data_raw = merged_data[(merged_data['_merge'] == 'left_only') & (~merged_data['InvoiceNo'].astype(str).str.startswith("C"))]

# Drop the indicator column
data_raw = data_raw.drop(columns=['_merge'])

In [ ]:
data_raw.head()

In [ ]:
# Selecting relevant features and calculating total sales
features = ['CustomerID', 'InvoiceNo', 'InvoiceDate', 'Quantity', 'UnitPrice', 'Country']
data = data_raw[features]
data['TotalSales'] = data['Quantity'].multiply(data['UnitPrice'])

# Removing transactions with missing customer IDs - they dont contribute to individual customer behaviour
data = data[data['CustomerID'].notna()]
data['CustomerID'] = data['CustomerID'].astype(int).astype(str)
print(data.shape)
data.head()

In [ ]:
# Check for any missing values
pd.DataFrame(zip(data.isnull().sum(), data.isnull().sum()/len(data)), columns=['Count', 'Proportion'], index=data.columns)

In [ ]:
# Summarize the dataset's content to understand its scope
maxdate = data['InvoiceDate'].dt.date.max()
mindate = data['InvoiceDate'].dt.date.min()
unique_cust = data['CustomerID'].nunique()
tot_quantity = data['Quantity'].sum()
tot_sales = data['TotalSales'].sum()

print(f"The time range of transactions is: {mindate} to {maxdate}")
print(f"Total number of unique customers: {unique_cust}")
print(f"Total quantity sold: {tot_quantity}")
print(f"Total sales for the period: {tot_sales}")

In [ ]:
# Prep a summary from transaction data, transforming individua transaction data into data at the customer level

data_summary_rfm = clv.utils.clv_summary(data, 'CustomerID', 'InvoiceDate', 'TotalSales')
data_summary_rfm = data_summary_rfm.rename(columns={'CustomerID': 'customer_id'})
data_summary_rfm.index = data_summary_rfm['customer_id']
data_summary_rfm

In [ ]:
data_summary_rfm = data_summary_rfm[data_summary_rfm['monetary_value'] > 0]

In [ ]:
# Plotting the frequency dist of customers
plt.figure(figsize=(10, 6))
plt.hist(data_summary_rfm['frequency'], bins=20, range=[0, 30], edgecolor='k', alpha=0.7)
plt.title('Frequency Distribution')
plt.xlabel('Frequency')
plt.ylabel('Number of Customers')
plt.grid(axis='y', alpha=0.6)
plt.show

one_time = round(sum(data_summary_rfm['frequency'] == 1) / float(len(data_summary_rfm)) * 100, 2)
print(f"Percentage of customers who purchased only once: {one_time}%")

In [ ]:
# Calculating Days between Purchases

# Select distinct CustomerID and InvoiceData
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])
unique_purchases = data[['CustomerID', 'InvoiceDate']].drop_duplicates()

# Sorting values to esure the calculation of the difference correctly
unique_purchases['NextInvoiceDate'] = unique_purchases.groupby('CustomerID')['InvoiceDate'].shift(-1)
unique_purchases['DaysBetween'] = (unique_purchases['NextInvoiceDate'] - unique_purchases['InvoiceDate']).dt.days

# Calculate average days
customer_avg_days = unique_purchases.groupby('CustomerID')['DaysBetween'].mean().dropna()

# Plotting the histogram
plt.figure(figsize=(10,6))
plt.hist(customer_avg_days, bins=25, edgecolor='k', alpha=0.7)
plt.title('Distribution of average days between purchases per customer')
plt.xlabel('Average number of days between purchases')
plt.ylabel('number of customers')
plt.grid(axis='y', alpha=0.6)
plt.show()

In [ ]:
data_summary_rfm

In [ ]:
# Initializing the BG/NBD model
bgm = clv.BetaGeoModel(
    data = data_summary_rfm,
)
bgm.build_model()
bgm